# Práctica 3 - Primera parte

Alejandra Gavino-Dias

### Importamos los paquetes necesarios

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree, export_text
import re as re

In [2]:
tabla = pd.DataFrame(columns=['Presion','Clase'], index=np.arange(6))

In [3]:
tabla.Presion=[40, 48, 60, 72, 80, 90]
tabla.Clase=['+', '+', '-', '-', '-', '+']

In [4]:
tabla

,Presion,Clase
0,40,+
1,48,+
2,60,-
3,72,-
4,80,-
5,90,+


In [5]:
tabla.Clase.value_counts(normalize=True)

Clase
+    0.5
-    0.5
Name: proportion, dtype: float64

### Calculamos la entropía

In [7]:
H = entropy(tabla.Clase.value_counts(normalize=True), base=2)
print("Valor de la entropía: ",H)

Valor de la entropía:  1.0


In [8]:
tabla1 = tabla.copy() #no puedo copiar la tabla directamente como si fuera copia = x
tabla1.Presion = np.digitize(tabla.Presion, bins=[(48+60)/2])
tabla1

,Presion,Clase
0,0,+
1,0,+
2,1,-
3,1,-
4,1,-
5,1,+


### Calculamos la ganancia para los dos umbrales

In [11]:
l1 = len(tabla1)
G1 = H
for v in tabla1.Presion.unique():
    Class1 = tabla1.loc[tabla1.Presion == v]
    G1 -= (len(Class1)/l1)*entropy(Class1.iloc[:, -1].value_counts(normalize=True), base=2)
print("Ganancia con (48+60)/2: ",G1)

Ganancia con (48+60)/2:  0.4591479170272448


In [10]:
tabla2 = tabla.copy() #no puedo copiar la tabla directamente como si fuera copia = x
tabla2.Presion = np.digitize(tabla.Presion, bins=[(80+90)/2])
tabla2

,Presion,Clase
0,0,+
1,0,+
2,0,-
3,0,-
4,0,-
5,1,+


In [12]:
l2 = len(tabla2)
G2 = H
for v in tabla2.Presion.unique():
    Class2 = tabla2.loc[tabla2.Presion == v]
    G2 -= (len(Class2)/l2)*entropy(Class2.iloc[:, -1].value_counts(normalize=True), base=2)
print("Ganancia con (80+90)/2: ",G2)

Ganancia con (80+90)/2:  0.19087450462110933


In [13]:
G1>G2

True

La ganancia con el primer umbral es mayor que la segunda ganancia, por eso nos quedaremos con: (48+60)/2

## Arbol de decisión

In [14]:
clf = DecisionTreeClassifier(criterion = "entropy", max_leaf_nodes = 2)

In [15]:
clf.fit(np.array(tabla.Presion).reshape(-1,1), tabla.Clase);

In [18]:
txt = export_text(clf, feature_names=['Presion'])
print(txt)

|--- Presion <= 54.00
|   |--- class: +
|--- Presion >  54.00
|   |--- class: -



***

In [22]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
thoracic_surgery_data = fetch_ucirepo(id=277) 
  
# data (as pandas dataframes) 
X = thoracic_surgery_data.data.features 
y = thoracic_surgery_data.data.targets 

In [23]:
tabla = X.assign(risk1yr=y)
tabla

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,risk1yr
0,DGN2,2.88,2.16,PRZ1,F,F,F,T,T,OC14,F,F,F,T,F,60,F
1,DGN3,3.40,1.88,PRZ0,F,F,F,F,F,OC12,F,F,F,T,F,51,F
2,DGN3,2.76,2.08,PRZ1,F,F,F,T,F,OC11,F,F,F,T,F,59,F
3,DGN3,3.68,3.04,PRZ0,F,F,F,F,F,OC11,F,F,F,F,F,54,F
4,DGN3,2.44,0.96,PRZ2,F,T,F,T,T,OC11,F,F,F,T,F,73,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,DGN2,3.88,2.12,PRZ1,F,F,F,T,F,OC13,F,F,F,T,F,63,F
466,DGN3,3.76,3.12,PRZ0,F,F,F,F,F,OC11,F,F,F,T,F,61,F
467,DGN3,3.04,2.08,PRZ1,F,F,F,T,F,OC13,F,F,F,F,F,52,F
468,DGN3,1.96,1.68,PRZ1,F,F,F,T,T,OC12,F,F,F,T,F,79,F


In [19]:
f = open('salida.txt')
contenido = f.read()
f.close()
print(contenido)

PRE9 = T
|   PRE11 = T: T (2.0)
|   PRE11 = F
|   |   PRE6 = PRZ2: T (2.0/1.0)
|   |   PRE6 = PRZ1
|   |   |   PRE10 = T
|   |   |   |   PRE7 = T: T (2.0/1.0)
|   |   |   |   PRE7 = F
|   |   |   |   |   PRE8 = T
|   |   |   |   |   |   DGN = DGN3: T (3.0/1.0)
|   |   |   |   |   |   DGN = DGN2: F (2.0)
|   |   |   |   |   |   DGN = DGN4: F (0.0)
|   |   |   |   |   |   DGN = DGN6: F (0.0)
|   |   |   |   |   |   DGN = DGN5: F (0.0)
|   |   |   |   |   |   DGN = DGN8: F (0.0)
|   |   |   |   |   |   DGN = DGN1: F (0.0)
|   |   |   |   |   PRE8 = F: F (11.0)
|   |   |   PRE10 = F: F (5.0)
|   |   PRE6 = PRZ0: T (4.0/1.0)
PRE9 = F
|   PRE17 = T
|   |   PRE8 = T: T (4.0/1.0)
|   |   PRE8 = F
|   |   |   AGE <= 65
|   |   |   |   PRE11 = T: T (3.0)
|   |   |   |   PRE11 = F
|   |   |   |   |   PRE14 = OC11: F (5.0)
|   |   |   |   |   PRE14 = OC14: T (1.0)
|   |   |   |   |   PRE14 = OC12
|   |   |   |   |   |   AGE <= 62: F (7.0/1.0)
|   |   |   |   |   |   AGE > 62: T (2.0)
|   |   |   |

In [20]:
no_usado=[]
for atr in tabla.columns[:-1]:
    if contenido.find(atr)==-1:
        no_usado.append(atr)
no_usado

['PRE19', 'PRE32']

In [21]:
tabla.drop(labels= no_usado, axis=1, inplace=True)
#si no ponemos el inplace fisicamente no ha hecho el borrado
#lo de axis es por columnas, si ponemos 0 lo hace por filas
tabla

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE25,PRE30,AGE,risk1yr
0,DGN2,2.88,2.16,PRZ1,F,F,F,T,T,OC14,F,F,T,60,F
1,DGN3,3.40,1.88,PRZ0,F,F,F,F,F,OC12,F,F,T,51,F
2,DGN3,2.76,2.08,PRZ1,F,F,F,T,F,OC11,F,F,T,59,F
3,DGN3,3.68,3.04,PRZ0,F,F,F,F,F,OC11,F,F,F,54,F
4,DGN3,2.44,0.96,PRZ2,F,T,F,T,T,OC11,F,F,T,73,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,DGN2,3.88,2.12,PRZ1,F,F,F,T,F,OC13,F,F,T,63,F
466,DGN3,3.76,3.12,PRZ0,F,F,F,F,F,OC11,F,F,T,61,F
467,DGN3,3.04,2.08,PRZ1,F,F,F,T,F,OC13,F,F,F,52,F
468,DGN3,1.96,1.68,PRZ1,F,F,F,T,T,OC12,F,F,T,79,F


In [22]:
tabla.to_csv('salida.csv', index=False)

## Resultados weka con atributos no usados

Correctly Classified Instances         123               76.875  %

Incorrectly Classified Instances        37               23.125  %

***

### Discretizacion de los valores numericos

#### Vemos que atributos son numericos

In [23]:
atrib_num = list(tabla._get_numeric_data().columns)

In [24]:
atrib_num

['PRE4', 'PRE5', 'AGE']

In [25]:
for atributo in atrib_num:
    b = []
    for pos in re.finditer(atributo+ ' <= ', contenido):
        if contenido[pos.end():].find(":") < contenido[pos.end():].find("\n"):
            segmento = contenido[pos.end():].find(":")
        else:
            segmento = contenido[pos.end():].find("\n")
        b.append(float(contenido[pos.end():pos.end()+segmento]))
        print(b[-1])
    c = list(set(b))
    c.sort(key=float)
    print(atributo, c)
    tabla[atributo] = [chr(65+i) for i in np.digitize(tabla[atributo], bins=c)]

2.88
2.66
PRE4 [2.66, 2.88]
2.04
PRE5 [2.04]
65.0
62.0
70.0
AGE [62.0, 65.0, 70.0]


In [26]:
tabla

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE25,PRE30,AGE,risk1yr
0,DGN2,C,B,PRZ1,F,F,F,T,T,OC14,F,F,T,A,F
1,DGN3,C,A,PRZ0,F,F,F,F,F,OC12,F,F,T,A,F
2,DGN3,B,B,PRZ1,F,F,F,T,F,OC11,F,F,T,A,F
3,DGN3,C,B,PRZ0,F,F,F,F,F,OC11,F,F,F,A,F
4,DGN3,A,A,PRZ2,F,T,F,T,T,OC11,F,F,T,D,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,DGN2,C,B,PRZ1,F,F,F,T,F,OC13,F,F,T,B,F
466,DGN3,C,B,PRZ0,F,F,F,F,F,OC11,F,F,T,A,F
467,DGN3,C,B,PRZ1,F,F,F,T,F,OC13,F,F,F,A,F
468,DGN3,A,A,PRZ1,F,F,F,T,T,OC12,F,F,T,D,F


In [27]:
tabla.to_csv('salida2.csv', index=False)

### Resultados weka con atributos numericos discretizados

Correctly Classified Instances         130               81.25   %

Incorrectly Classified Instances        30               18.75   %